In [1]:
import os
from nltk import word_tokenize
import re,string
import time
import numpy as np
from getProcess import parsecorpus, dataProcessing

Create a parser. Cleaning the text and create data iterator

In [2]:
path1 = 'en_US.news.txt'
parser = parsecorpus()
myIterator = dataProcessing(parser, corpus_path = path1, filtered_path = 'news_filtered.txt', 
                            filter_size = 0.03, n_gram = 2)

Volcabulary size is 63579


Number of bi-grams we have:

In [3]:
parser.doc_size

30307

In [4]:
import tensorflow as tf

In [5]:
g = tf.Graph()
n_words = parser.v_count        # Number of possible words
n_hidden = 500                 # Number of hidden layer nodes
with g.as_default():
    tf_x = tf.placeholder(tf.float32, shape = (None, n_words), name = 'center_word_vec')

    tf_y = tf.placeholder(tf.int32, shape = (None, n_words), name = 'labels')

    h1 = tf.layers.dense(inputs = tf_x, units = n_hidden, activation = None, name = 'hidden_1')

    h2 = tf.layers.dense(inputs = h1, units = n_words, activation = None, name = 'logits')
    
    print (tf_x, h2)

    predictions = {'probabilities': tf.nn.softmax(h2, name = 'probabilities'), 'label': tf.argmax(h2, axis = 1, name = 'word_index')}
    
    loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels = tf_y, logits = h2), name='cross_entropy_loss')

    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)

    train_op = optimizer.minimize(loss)

    init_op = tf.global_variables_initializer()

Tensor("center_word_vec:0", shape=(?, 63579), dtype=float32) Tensor("logits/BiasAdd:0", shape=(?, 63579), dtype=float32)


In [ ]:
num_words = 26000
with tf.Session(graph = g) as sess:
    saver = tf.train.Saver()
    sess.run(init_op)
    for idx in range(num_words):
        if idx > 0 and idx % 2000 == 0:
            print ('training with %d -th word...' % idx)
        x, y = next(myIterator)
        feed = {tf_x: x.reshape(-1, n_words), tf_y: y.reshape(-1, n_words)}
        cost, _ = sess.run([loss, train_op], feed_dict = feed)
    saver.save(sess, './my_model.ckpt')

Testing and calculating prediction accuracy

In [ ]:
g2 = tf.Graph()
test_size = 5000
y_test = []
x_test = []
with tf.Session(graph = g) as sess:
    saver = tf.train.import_meta_graph('my_model.ckpt.meta')
    saver.restore(sess, 'my_model.ckpt')
    preds = []
    for i in range(test_size):
        x_t, y_t = next(myIterator)
        y_test.append(y_t)
        x_test.append(x_t)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    feed = {'center_word_vec:0': x_test.reshape(-1, n_words)}
    preds.append(sess.run('logits/BiasAdd:0', feed_dict = feed))
    preds = np.array(preds)

In [ ]:
preds = preds.reshape((-1, parser.v_count))
predictions = np.argmax(preds, axis = 1)

In [ ]:
y_test.reshape((-1, parser.v_count))
y_test = np.argmax(y_test, axis = 1)

In [ ]:
np.mean(predictions==y_test)